In [ ]:
import numpy as np
import itertools
import operator
import functools
import time
import traceback

binary_dict = {
    0: [0, 0, 0], 1: [0, 0, 1], 2: [0, 1, 0], 3: [0, 1, 1],
    4: [1, 0, 0], 5: [1, 0, 1], 6: [1, 1, 0], 7: [1, 1, 1]
}

def convert_to_binary(array_octal_digits):
    if not hasattr(array_octal_digits, '__iter__'):
        print(f"Ошибка convert_to_binary: вход не итерируемый - {array_octal_digits}")
        return []
    binary_list = []
    for element in array_octal_digits:
        bits = binary_dict.get(element)
        if bits is None:
            # print(f"Ошибка convert_to_binary: некорректная октальная цифра {element}")
            raise ValueError(f"Некорректная октальная цифра {element} в {array_octal_digits}")
        binary_list.extend(bits)
    return binary_list

def binary_array_to_decimal(arr_binary_triplet):
    if len(arr_binary_triplet) == 3 and all(bit in [0, 1] for bit in arr_binary_triplet):
        binary_str = ''.join(map(str, arr_binary_triplet))
        return int(binary_str, 2)
    else:
        # print(f"Ошибка binary_array_to_decimal: вход не бинарная триада - {arr_binary_triplet}")
        raise ValueError(f"Input must be an array of 3 elements containing 0 or 1: {arr_binary_triplet}")

def transform_array_to_binary(array_of_octal_rows):
    transformed_array = []
    if len(array_of_octal_rows) == 0:
        return []
    for row in array_of_octal_rows:
        if len(row) == 0: 
             transformed_array.append([])
             continue
        try:
            binary_row = convert_to_binary(row)
            transformed_array.append(binary_row)
        except ValueError as e:
            # print(f"Ошибка в transform_array_to_binary при обработке строки {row}: {e}")
            raise e
    return transformed_array
    
def binary_to_decimal_array(flat_binary_list):
    if len(flat_binary_list) % 3 != 0:
        # print(f"Ошибка binary_to_decimal_array: длина бинарного списка не кратна 3 - {len(flat_binary_list)}")
        raise ValueError("Длина бинарного списка должна быть кратна 3")
    
    transformed = []
    for i in range(0, len(flat_binary_list), 3):
        triplet = flat_binary_list[i:i+3]
        found_key = None
        for key, value in binary_dict.items():
            if value == list(triplet):
                transformed.append(key)
                found_key = True
                break
        if not found_key:
            raise ValueError(f"Не удалось преобразовать биты {triplet} в октальное число.")
    return transformed

def G(m, n):
    A = binary_dict[m]
    B = binary_dict[n]
    C = []
    for i in range(3):
        C.append(A[(i + 1) % 3] * B[(i + 2) % 3])
    return C

def F(m, n, t):
    A = binary_dict[m]
    B = binary_dict[n]
    C_val = binary_dict[t]
    D = []
    for i in range(3):
        D.append(A[(i + 1) % 3] * B[(i + 2) % 3] * C_val[i])
    return D

G_dict_global = {}
F_dict_global = {}

def precompute_dicts():
    global G_dict_global, F_dict_global
    print("INFO: Предварительный расчет словарей G_dict и F_dict...")
    for m_g in range(8):
        for n_g in range(8):
            G_dict_global[(m_g, n_g)] = binary_array_to_decimal(G(m_g, n_g))
    for m_f in range(8):
        for n_f in range(8):
            for q_f in range(8):
                F_dict_global[(m_f, n_f, q_f)] = binary_array_to_decimal(F(m_f, n_f, q_f))
    print("INFO: Словари G_dict и F_dict рассчитаны.")

class ArrayWithRelations:
    def __init__(self, solutions_x12):
        if isinstance(solutions_x12, np.ndarray):
            self.array = solutions_x12
            num_solutions = solutions_x12.shape[0]
        elif isinstance(solutions_x12, list):
            if not solutions_x12:
                self.array = np.array([])
                num_solutions = 0
            elif solutions_x12 and isinstance(solutions_x12[0], (list, np.ndarray)):
                 self.array = np.array(solutions_x12) 
            else: 
                 self.array = solutions_x12
            num_solutions = len(solutions_x12)
        else:
            raise TypeError("solutions_x12 должен быть NumPy array или списком.")

        self._relations = {i: [] for i in range(num_solutions)}

    def get_array_by_index(self, index):
        if not (0 <= index < len(self.array)):
            # raise IndexError(f"Индекс x12 ({index}) (get_array_by_index) выходит за пределы.")
            print(f"WARN: Индекс x12 ({index}) (get_array_by_index) выходит за пределы.")
            return None
        return self.array[index]

    def add_related_array(self, x12_idx, x23_solution_list):
        if not (0 <= x12_idx < len(self.array)):
            print(f"WARN: Индекс x12 ({x12_idx}) (add_related_array) выходит за пределы.")
            return
        
        self._relations[x12_idx] = [] 
        for x23_sol_arr in x23_solution_list:
            self._relations[x12_idx].append({
                'array': x23_sol_arr,
                'sub_related': [] 
            })

    def get_related_arrays(self, x12_idx):
        if not (0 <= x12_idx < len(self.array)):
            return [] 
        return [x23_entry['array'] for x23_entry in self._relations.get(x12_idx, [])]

    def add_sub_related_array(self, x12_idx, x23_entry_idx, x31_solution_list):
        if not (0 <= x12_idx < len(self.array)):
            print(f"WARN (add_sub_related_array): Индекс x12 ({x12_idx}) выходит за пределы.")
            return
        
        x23_entries_list = self._relations.get(x12_idx)
        if x23_entries_list is None or not (0 <= x23_entry_idx < len(x23_entries_list)):
            print(f"WARN (add_sub_related_array): Индекс x23_entry ({x23_entry_idx}) для x12 ({x12_idx}) выходит за пределы.")
            return

        current_x23_entry = x23_entries_list[x23_entry_idx]
        current_x23_entry['sub_related'] = [] 
        if x31_solution_list: 
            for x31_sol_arr in x31_solution_list:
                current_x23_entry['sub_related'].append({
                    'array_x31': x31_sol_arr,
                    'x13_solutions_list': []
                })

    def get_sub_related_arrays(self, x12_idx, x23_entry_idx):
        if not (0 <= x12_idx < len(self.array)):
            return []
        
        x23_entries_list = self._relations.get(x12_idx)
        if x23_entries_list is None or not (0 <= x23_entry_idx < len(x23_entries_list)):
            return []
        
        return x23_entries_list[x23_entry_idx].get('sub_related', [])

    def add_x13_solutions(self, x12_idx, x23_entry_idx, x31_entry_idx, x13_list):
        x31_entries_list = self.get_sub_related_arrays(x12_idx, x23_entry_idx)
        
        if not (0 <= x31_entry_idx < len(x31_entries_list)):
            print(f"WARN (add_x13_solutions): Индекс x31_entry ({x31_entry_idx}) выходит за пределы.")
            return
        
        current_x31_entry = x31_entries_list[x31_entry_idx]
        current_x31_entry['x13_solutions_list'] = x13_list

    def get_x13_solutions(self, x12_idx, x23_entry_idx, x31_entry_idx):
        x31_entries_list = self.get_sub_related_arrays(x12_idx, x23_entry_idx)
        if not (0 <= x31_entry_idx < len(x31_entries_list)):
            return []
        return x31_entries_list[x31_entry_idx].get('x13_solutions_list', [])


In [ ]:
def create_linear_system_for_12(matrix):
    """
    Строит бинарную систему уравнений для группы «12».
    Для каждой пары строк (i,j) исходной 3×7-матрицы берётся G_dict,
    формируя 9 строк октальных значений, которые конвертируются в
    бинарный формат (9×21 бит).
    Args:
        matrix (np.ndarray): Массив формы (3,7) со значениями 0..7.
    Returns:
        np.ndarray: Двоичный массив формы (9,21) dtype=int.
    """
    system = []
    for i in range (0, 3):
        for j in range (0,3):
            for p in range (0,7):
                system.append(G_dict_global[(matrix[i][p], matrix[j][p])])

    reshaped_system = []
    for i in range(0, len(system), 7):
        reshaped_system.append(system[i:i+7])
    
    return transform_array_to_binary(np.array(reshaped_system, dtype=int))

def find_nontrivial_solutions(A, f):
    """
    Находит частное и базисные решения ядра системы A*x = f в GF(2).
    Args:
        A: SageMath Matrix в поле GF(2).
        f: SageMath vector — правая часть системы.
    Returns:
        List[vector]: Список нетривиальных решений (частное + комбинации базисных векторов).
    """
    # Найдем частное решение
    x0 = A.solve_right(f)
    
    # Найдем базис правого ядра
    kernel_basis = A.right_kernel().basis()
    
    n = len(kernel_basis)

    # Список для хранения решений
    solutions = []
    
    # Добавляем частное решение только если оно ненулевое
    if x0 != vector(GF(2), [0] * A.ncols()):
        solutions.append(x0)

    # Перебираем все возможные линейные комбинации базисных векторов ядра
    if n > 0:
        for i in range(1, 2**n):  # начиная с 1, исключаем тривиальное решение (0)
            # Представляем число i в двоичной системе для получения коэффициентов комбинации
            coeffs = [int(b) for b in bin(i)[2:].zfill(n)]
            
            # Строим линейную комбинацию базисных векторов с коэффициентами coeffs
            z = sum(c * k for c, k in zip(coeffs, kernel_basis))
            
            # Вычисляем решение как x = x0 + z
            solution = x0 + z
            
            # Пропускаем нулевое решение
            if solution != vector(GF(2), [0] * A.ncols()):
                solutions.append(solution)

    transformed_solutions = []
    for solution in solutions:
        transformed = binary_to_decimal_array(list(solution))
        transformed_solutions.append(transformed)
    transformed_solutions_array = np.array(transformed_solutions, dtype=int)

    return transformed_solutions_array      


def create_four_equation_for_12(matrix, nontrivial_solutions, output_file):
    valid_solutions = []
    
    for k in range(0, len(nontrivial_solutions)):
        system = []
        
        # ii, 12, 12
        for i in range(0, 3):
            for p in range(0, 7):
                system.append(F_dict_global[matrix[i][p], nontrivial_solutions[k][p], nontrivial_solutions[k][p]])
        
        # 12, 12, 12
        for p in range(0, 7):
            system.append(F_dict_global[nontrivial_solutions[k][p], nontrivial_solutions[k][p], nontrivial_solutions[k][p]])
        
        new_matrix = []
        for i in range(0, len(system), 7):
            row = system[i:i+7]
            new_matrix.append(sum(convert_to_binary(row)) % 2)
        
        if new_matrix == [0, 0, 0, 0]:
            valid_solutions.append(nontrivial_solutions[k])
    
    if output_file:
        np.save(output_file, valid_solutions)
    
    return valid_solutions

def create_linear_system_for_23(first_9, solutions, matrix, t):
    system = []
    for i in range(0, 3):  # ii 12
        for p in range(0, 7):
            system.append(G_dict_global[(matrix[i][p], solutions[t][p])])

    for i in range(0, 3):  # 12 ii
        for p in range(0, 7):
            system.append(G_dict_global[(solutions[t][p], matrix[i][p])])

    for p in range(0, 7):  # 12 12
        system.append(G_dict_global[(solutions[t][p], solutions[t][p])])

    new_matrix = []
    for i in range(0, len(system), 7):
        row = system[i:i+7]
        new_matrix.append(row)

    binary_matrix = transform_array_to_binary(new_matrix)
    first_9 = np.vstack([first_9, binary_matrix])
    return first_9



def create_four_equation_for_23(matrix, nontrivial_solutions, solution_12):
    valid_solutions = []  # Список для хранения решений, которые соответствуют условию
    
    for k in range(0, len(nontrivial_solutions)):
        system = []
        
        # ii, 23, 23
        for i in range(0, 3):
            for p in range(0, 7):
                system.append(F_dict_global[matrix[i][p], nontrivial_solutions[k][p], nontrivial_solutions[k][p]])
        
        # 12 23 23
        for p in range(0, 7):
            system.append(F_dict_global[solution_12[p], nontrivial_solutions[k][p], nontrivial_solutions[k][p] ])

        # 23, 23, 23
        #for p in range(0, 7):
         #   system.append(F_dict[nontrivial_solutions[k][p], nontrivial_solutions[k][p], nontrivial_solutions[k][p]])
        
        new_matrix = []
        for i in range(0, len(system), 7):
            row = system[i:i+7]
            new_matrix.append(sum(convert_to_binary(row)) % 2)

        if new_matrix == [0, 0, 0, 0]:
            valid_solutions.append(nontrivial_solutions[k])

    return valid_solutions


def create_linear_system_for_31(system_for_23_binary, solution_12, solutions_23, matrix):
    """
    Создает бинарную СЛУ для элемента '31'.
    Принимает БИНАРНУЮ систему для '23', вычисляет новые строки,
    преобразует их в бинарный вид и добавляет к входной системе.
    """
    new_rows_octal_flat = [] 
    # ii 23 
    for i in range(0, 3):
        for p in range(0, 7):
            new_rows_octal_flat.append(G_dict_global[(matrix[i][p], solutions_23[p])])
    # 23 ii
    for i in range(0, 3):
        for p in range(0, 7):
            new_rows_octal_flat.append(G_dict_global[(solutions_23[p], matrix[i][p])])
    # 23 23
    for p in range(0, 7):
        new_rows_octal_flat.append(G_dict_global[(solutions_23[p], solutions_23[p])])
    # 23 12
    for p in range(0, 7):
        new_rows_octal_flat.append(G_dict_global[(solutions_23[p], solution_12[p])])
    # 12 23 (Эта часть соответствует f=[...1] в неоднородной системе)
    for p in range(0, 7):
        new_rows_octal_flat.append(G_dict_global[(solution_12[p], solutions_23[p])])

    new_matrix_octal = []
    num_new_rows = 9
    if len(new_rows_octal_flat) != num_new_rows * 7:
         raise ValueError(f"Ожидалось {num_new_rows * 7} октальных значений для новых строк, получено {len(new_rows_octal_flat)}")
         
    for i in range(0, len(new_rows_octal_flat), 7):
        row = new_rows_octal_flat[i:i+7]
        new_matrix_octal.append(row)

    new_rows_binary = transform_array_to_binary(new_matrix_octal) 

    system_for_31_combined_binary = np.vstack([np.array(system_for_23_binary, dtype=int), 
                                               np.array(new_rows_binary, dtype=int)])
    
    return system_for_31_combined_binary

def create_five_equation_for_31(matrix, solutions_23, solution_12, nontrivial_solutions_for_31):
    valid_solutions = [] 
    print(f"  [DEBUG] Запущена create_five_equation_for_31. Кандидатов: {len(nontrivial_solutions_for_31)}")

    candidates_to_check = nontrivial_solutions_for_31 

    for r in range(len(candidates_to_check)): 
        candidate = candidates_to_check[r] 
        print(f"\n  [DEBUG] Проверка кандидата r={r}: {candidate}") 

        system_checks_octal_rows = []
        # Блок 1: Проверки ii, 31, 31 (i=0, 1, 2)
        print(f"    Проверки 0, 1, 2 (ii, 31, 31)")
        for i in range(0, 3):
            row_octal = []
            # print(f"      Проверка {i}:") # Доп. отладка, если нужно
            for p in range(0, 7):
                m_arg = matrix[i][p]
                n_arg = candidate[p]
                t_arg = candidate[p]
                # print(f"        p={p}: F({m_arg}, {n_arg}, {t_arg})")
                f_result = F_dict[(m_arg, n_arg, t_arg)]
                row_octal.append(f_result)
            system_checks_octal_rows.append(row_octal)
            print(f"      Проверка {i}: Октал строка = {row_octal}")

        # Блок 2: Проверка 12, 31, 31
        print(f"    Проверка 3 (12, 31, 31)")
        row_octal = []
        for p in range(0, 7):
            m_arg = solution_12[p]
            n_arg = candidate[p]
            t_arg = candidate[p]
            # print(f"        p={p}: F({m_arg}, {n_arg}, {t_arg})")
            f_result = F_dict[(m_arg, n_arg, t_arg)]
            row_octal.append(f_result)
        system_checks_octal_rows.append(row_octal)
        print(f"      Проверка 3: Октал строка = {row_octal}")

        # Блок 3: Проверка 23, 31, 31
        print(f"    Проверка 4 (23, 31, 31)")
        row_octal = []
        for p in range(0, 7):
            m_arg = solutions_23[p]
            n_arg = candidate[p]
            t_arg = candidate[p]
            # print(f"        p={p}: F({m_arg}, {n_arg}, {t_arg})") 
            f_result = F_dict[(m_arg, n_arg, t_arg)]
            row_octal.append(f_result)
        system_checks_octal_rows.append(row_octal)
        print(f"      Проверка 4: Октал строка = {row_octal}")

        check_results = []
        print(f"     Результаты 5 проверок для r={r}") 
        for i, row_oct in enumerate(system_checks_octal_rows):
            #try: 
               binary_row_flat = convert_to_binary(row_oct) 
               sum_bits = sum(binary_row_flat)
               check_value = sum_bits % 2      
               #  Печать 3: Результат конкретной проверки
               print(f"    [DEBUG] Проверка {i}: СуммаБит={sum_bits} -> Результат={check_value}") 
               check_results.append(check_value)
            #except Exception as e_conv:
             #  print(f"    [DEBUG] Ошибка при конвертации/сумме для проверки {i}, строка={row_oct}: {e_conv}")
              # check_results.append(-1) # Маркер ошибки

        #  Печать 4: Итоговый вектор проверок 
        print(f"    [DEBUG] Итоговый вектор проверок для r={r}: {check_results}") 

        if check_results == [0, 0, 0, 0, 0]:
             print(f"    [DEBUG] Кандидат r={r} ПРОШЕЛ проверку!")
             valid_solutions.append(candidate) 

    print(f"  [DEBUG] Завершена create_five_equation_for_31. Найдено валидных: {len(valid_solutions)}")
    return valid_solutions

def create_linear_system_for_13(system_for_31_binary, solution_12, solution_23, solution_31, matrix):
    print(f"    [DEBUG_CREATE_13] Входные данные для create_linear_system_for_13:")
    print(f"    [DEBUG_CREATE_13] solution_12: {solution_12} (тип: {type(solution_12)})")
    print(f"    [DEBUG_CREATE_13] solution_23: {solution_23} (тип: {type(solution_23)})")
    print(f"    [DEBUG_CREATE_13] solution_31: {solution_31} (тип: {type(solution_31)})")
    print(f"    [DEBUG_CREATE_13] matrix[0]: {matrix[0] if len(matrix) > 0 else 'Matrix пуста или некорректна'}") # Печатаем первую строку для примера
    try:
        new_rows_octal_flat = [] 
        # ii 31 
        for i in range(0, 3):
            for p in range(0, 7):
                new_rows_octal_flat.append(G_dict_global[(matrix[i][p], solution_31[p])])
        # 31 ii
        for i in range(0, 3):
            for p in range(0, 7):
                new_rows_octal_flat.append(G_dict_global[(solution_31[p], matrix[i][p])])
        # 31 31
        for p in range(0, 7):
            new_rows_octal_flat.append(G_dict_global[(solution_31[p], solution_31[p])])
        # 31 12
        for p in range(0, 7):
            new_rows_octal_flat.append(G_dict_global[(solution_31[p], solution_12[p])])
        # 12 31
        for p in range(0, 7):
            new_rows_octal_flat.append(G_dict_global[(solution_12[p], solution_31[p])])
        # 31 23
        for p in range(0, 7):
            new_rows_octal_flat.append(G_dict_global[(solution_31[p], solution_23[p])])
        # 23 31
        for p in range(0, 7):
            new_rows_octal_flat.append(G_dict_global[(solution_23[p], solution_31[p])])
    except Exception as e:
        print(f"    ! Неожиданная ошибка при добавлении строк: {type(e).__name__} - {e}")


    try:
        new_matrix_octal = []
        num_new_rows = 11 
        if len(new_rows_octal_flat) != num_new_rows * 7:
            raise ValueError(f"Ожидалось {num_new_rows * 7} октальных значений для новых строк, получено {len(new_rows_octal_flat)}")
            
        for i in range(0, len(new_rows_octal_flat), 7):
            row = new_rows_octal_flat[i:i+7]
            new_matrix_octal.append(row)
    except Exception as e:
        print(f"    ! Неожиданная ошибка при построении матрицы: {type(e).__name__} - {e}")


    new_rows_binary = transform_array_to_binary(new_matrix_octal) 

    system_for_31_combined_binary = np.vstack([np.array(system_for_31_binary, dtype=int), 
                                               np.array(new_rows_binary, dtype=int)])
    
    return system_for_31_combined_binary


def create_six_equation_for_13(matrix, solutions_31, solutions_23, solutions_12, nontrivial_solutions_for_13):
    valid_solutions = [] 
    print(f"  [DEBUG] Запущена create_six_equation_for_13. Кандидатов: {len(nontrivial_solutions_for_13)}")

    candidates_to_check = nontrivial_solutions_for_13

    for r in range(len(candidates_to_check)): 
        candidate = candidates_to_check[r] 
        #  Печать 1: Сам кандидат
        print(f"\n  [DEBUG] Проверка кандидата r={r}: {candidate}") 

        system_checks_octal_rows = []
        # Блок 1: Проверки ii, 13, 13 (i=0, 1, 2)
        print(f"    Проверки 0, 1, 2 (ii, 13, 13)")
        for i in range(0, 3):
            row_octal = []
            # print(f"      Проверка {i}:") # Доп. отладка, если нужно
            for p in range(0, 7):
    
                m_arg = matrix[i][p]
                n_arg = candidate[p]
                t_arg = candidate[p]
                # print(f"        p={p}: F({m_arg}, {n_arg}, {t_arg})")
                f_result = F_dict_global[(m_arg, n_arg, t_arg)]
                row_octal.append(f_result)
            system_checks_octal_rows.append(row_octal)
            print(f"      Проверка {i}: Октал строка = {row_octal}")

        # Блок 2: Проверка 12, 13, 13
        print(f"    Проверка 3 (12, 13, 13)")
        row_octal = []
        for p in range(0, 7):
            m_arg = solutions_12[p]
            n_arg = candidate[p]
            t_arg = candidate[p]
            # print(f"        p={p}: F({m_arg}, {n_arg}, {t_arg})")
            f_result = F_dict_global[(m_arg, n_arg, t_arg)]
            row_octal.append(f_result)
        system_checks_octal_rows.append(row_octal)
        print(f"      Проверка 3: Октал строка = {row_octal}")

        # Блок 3: Проверка 23, 13, 13
        print(f"    Проверка 4 (23, 13, 13)")
        row_octal = []
        for p in range(0, 7):
            m_arg = solutions_23[p]
            n_arg = candidate[p]
            t_arg = candidate[p]
            # print(f"        p={p}: F({m_arg}, {n_arg}, {t_arg})")
            f_result = F_dict_global[(m_arg, n_arg, t_arg)]
            row_octal.append(f_result)
        system_checks_octal_rows.append(row_octal)
        print(f"      Проверка 4: Октал строка = {row_octal}")

        # Блок 4: Проверка 31, 13, 13
        print(f"    Проверка 5 (31, 13, 13)")
        row_octal = []
        for p in range(0, 7):
            m_arg = solutions_31[p]
            n_arg = candidate[p]
            t_arg = candidate[p]
            # print(f"        p={p}: F({m_arg}, {n_arg}, {t_arg})")
            f_result = F_dict_global[(m_arg, n_arg, t_arg)]
            row_octal.append(f_result)
        system_checks_octal_rows.append(row_octal)
        print(f"      Проверка 5: Октал строка = {row_octal}")

        check_results = []
        print(f"    Результаты 6 проверок для r={r}") 
        for i, row_oct in enumerate(system_checks_octal_rows):
            #try: 
               binary_row_flat = convert_to_binary(row_oct) 
               sum_bits = sum(binary_row_flat)
               check_value = sum_bits % 2      
               print(f"    [DEBUG] Проверка {i}: СуммаБит={sum_bits} -> Результат={check_value}") 
               check_results.append(check_value)
            #except Exception as e_conv:
             #  print(f"    [DEBUG] Ошибка при конвертации/сумме для проверки {i}, строка={row_oct}: {e_conv}")
              # check_results.append(-1)

        print(f"    [DEBUG] Итоговый вектор проверок для r={r}: {check_results}") 

        if check_results == [0, 0, 0, 0, 0, 0]:
             print(f"    [DEBUG] Кандидат r={r} ПРОШЕЛ проверку!")
             valid_solutions.append(candidate) 

    print(f"  [DEBUG] Завершена create_six_equation_for_13. Найдено валидных: {len(valid_solutions)}")
    return valid_solutions


In [ ]:
# Функции генерации "дополнений" и фильтрации (согласно последним уточнениям)

def generate_next_complement_matrix_octal(current_complement_flat_octal=None):
    """
    Генерирует следующую 3x4 октальную матрицу "дополнения" инкрементально.
    3x4 матрица = 12 октальных цифр.
    Возвращает (строки_матрицы_дополнения, плоский_список_для_след_итерации) или (None, None).
    """
    num_digits_in_complement = 3 * 4  # 12
    if current_complement_flat_octal is None:
        next_flat_octal = [0] * num_digits_in_complement
    else:
        next_flat_octal = list(current_complement_flat_octal)
        i = num_digits_in_complement - 1
        while i >= 0:
            next_flat_octal[i] += 1
            if next_flat_octal[i] < 8:
                break
            else:
                next_flat_octal[i] = 0
                if i == 0:
                    return None, None
                i -= 1
        if i < 0: return None, None

    new_complement_matrix_rows = []
    for r_idx in range(0, num_digits_in_complement, 4): 
        new_complement_matrix_rows.append(next_flat_octal[r_idx : r_idx + 4])
    return new_complement_matrix_rows, next_flat_octal

def generate_next_complement_matrix(current_complement_flat_octal=None, max_zero_cols=1):
    while True:
        matrix, next_flat = generate_next_complement_matrix_octal(current_complement_flat_octal)
        if matrix is None:
            return None, None
        if count_zero_columns(matrix) <= max_zero_cols:
            return matrix, next_flat
    
def count_zero_columns(matrix):
    """Подсчитывает количество полностью нулевых столбцов в матрице 3x7"""
    return sum(1 for col in zip(*matrix) if all(x == 0 for x in col))

def check_octal_column_has_binary_zero_column(octal_column_values):
    """
    Проверяет, содержит ли данный октальный столбец (список из 3 октальных чисел)
    хотя бы один нулевой столбец в своем 3x3 бинарном представлении.
    """
    if len(octal_column_values) != 3:
        return False

    binary_representation_3x3 = []
    for octal_digit in octal_column_values:
        bits = binary_dict.get(octal_digit)
        if bits is None: return False
        binary_representation_3x3.append(bits)
    
    for j_bin_col_idx in range(3):
        if all(binary_representation_3x3[i_bin_row_idx][j_bin_col_idx] == 0 for i_bin_row_idx in range(3)):
            return True
    return False

def count_binary_zero_property_in_given_columns(list_of_octal_columns):
    """
    Подсчитывает, сколько из предоставленных октальных столбцов
    (каждый столбец - список из 3 октальных цифр)
    имеют свойство "внутреннего бинарного нулевого столбца".
    """
    num_true_zero_property_columns = 0
    for oct_col_values in list_of_octal_columns:
        if check_octal_column_has_binary_zero_column(oct_col_values):
            num_true_zero_property_columns += 1
    return num_true_zero_property_columns

def analyze_4_complement_column_structures(list_of_4_column_tuples):
    """
    Анализирует список из 4 столбцов (кортежей) на предмет структуры.
    Возвращает словарь с флагами структуры и подсчетом каждого столбца.
    """
    stats = {
        'is_AAAA': False, 'is_AAAB': False, 'is_AABB': False,
        'is_AABC': False, 'is_ABCD': False,
        'column_counts': {}, 
        'distinct_columns': []
    }
    if len(list_of_4_column_tuples) != 4:
        return stats

    counts = {}
    for col_tuple in list_of_4_column_tuples:
        counts[col_tuple] = counts.get(col_tuple, 0) + 1
    
    stats['column_counts'] = counts
    stats['distinct_columns'] = list(counts.keys())
    num_distinct_cols = len(counts)
    counts_values_sorted = sorted(list(counts.values()), reverse=True)

    if num_distinct_cols == 1:
        stats['is_AAAA'] = True
    elif num_distinct_cols == 2:
        if counts_values_sorted == [3, 1]:
            stats['is_AAAB'] = True
        elif counts_values_sorted == [2, 2]:
            stats['is_AABB'] = True
    elif num_distinct_cols == 3:
        if counts_values_sorted == [2, 1, 1]:
            stats['is_AABC'] = True
    elif num_distinct_cols == 4:
        stats['is_ABCD'] = True
    return stats

def check_conditions_on_complement_cols_v7(full_matrix_octal):
    """
    Применяет уточненные условия C1_доп, C2_доп, C3_доп ТОЛЬКО к столбцам 3, 4, 5, 6
    полной матрицы full_matrix_octal (3x7).
    Возвращает строку с первым совпавшим условием или None.
    """
    if not isinstance(full_matrix_octal, list) or len(full_matrix_octal) != 3 or \
       not all(isinstance(row, list) and len(row) == 7 for row in full_matrix_octal):
        return "INVALID_SHAPE"

    complement_columns_list_of_lists = []
    for j_col_idx in range(3, 7): # Столбцы с индексами 3, 4, 5, 6
        column = [full_matrix_octal[i_row_idx][j_col_idx] for i_row_idx in range(3)]
        complement_columns_list_of_lists.append(column)
    
    complement_columns_as_tuples = [tuple(col) for col in complement_columns_list_of_lists]
    structure_stats = analyze_4_complement_column_structures(complement_columns_as_tuples)

    # Условие C1_доп: "A,A,B,B или A,A,A,A" (в 4 столбцах дополнения)
    if structure_stats['is_AABB'] or structure_stats['is_AAAA']:
        return "C1_доп_AABB_или_AAAA"

    # Условие C2_доп: "A,A И два столбца с бинарным нулем из оставшихся двух" (в 4 столбцах дополнения)
    # Это структура AABC, где B и C являются "нулевыми в бинарном представлении".
    if structure_stats['is_AABC']:
        column_counts = structure_stats['column_counts']
        column_A_tuple = None
        single_occurrence_column_lists_for_C2 = []

        for col_tuple, count in column_counts.items():
            if count == 2:
                column_A_tuple = col_tuple 
                break
        
        if column_A_tuple is not None:
            for original_col_list in complement_columns_list_of_lists:
                if tuple(original_col_list) != column_A_tuple:
                    is_already_added = False
                    for added_col in single_occurrence_column_lists_for_C2:
                        if added_col == original_col_list:
                            is_already_added = True
                            break
                    if not is_already_added:
                         single_occurrence_column_lists_for_C2.append(original_col_list)
            
            if len(single_occurrence_column_lists_for_C2) == 2:
                col_B_octal_list = single_occurrence_column_lists_for_C2[0]
                col_C_octal_list = single_occurrence_column_lists_for_C2[1]
                
                if check_octal_column_has_binary_zero_column(col_B_octal_list) and \
                   check_octal_column_has_binary_zero_column(col_C_octal_list):
                    return "C2_доп_AA_плюс_2_БИН_НУЛ_из_оставшихся"
    
    # Условие C3_доп: "Все 4 столбца дополнения являются нулевыми в бинарном представлении"
    num_binary_zero_in_complement = count_binary_zero_property_in_given_columns(complement_columns_list_of_lists)
    if num_binary_zero_in_complement == 4:
        return "C3_доп_Все_4_ДОП_СТОЛБЦА_БИН_НУЛ"
            
    return None


def run_pipeline_for_one_matrix(input_matrix_octal):
    """
    Запускает полный конвейер для поиска x_13 для данной input_matrix_octal.
    Возвращает кортеж: (True, словарь_с_решениями) если x13 найден,
                     (False, diagnostic_message) если не найден или произошла ошибка.
    """
    print(f"\n Начинаем обработку матрицы: {input_matrix_octal} ")
    
    try:
        print("\nШаг 1: Поиск решений x_12...")
        system_12_binary = create_linear_system_for_12(input_matrix_octal)
        A_12 = Matrix(GF(2), system_12_binary)
        f_12 = vector(GF(2), [0] * A_12.nrows())
        
        nontrivial_sols_12_arr = find_nontrivial_solutions(A_12, f_12)
        print(f"Найдено нетривиальных решений x_12: {len(nontrivial_sols_12_arr)}")
        
        valid_sols_12 = create_four_equation_for_12(input_matrix_octal, nontrivial_sols_12_arr, output_file=None)
        print(f"Валидных решений x_12 после проверки: {len(valid_sols_12)}")
        
        relations = ArrayWithRelations(np.array(valid_sols_12))

        print("\nШаг 2: Поиск решений x_23...")
        
        for x12_idx, current_sol_12 in enumerate(valid_sols_12):
            system_23_binary = create_linear_system_for_23(
                system_12_binary, valid_sols_12, input_matrix_octal, x12_idx
            )
            A_23 = Matrix(GF(2), system_23_binary)
            f_23 = vector(GF(2), [0] * A_23.nrows())
            
            nontrivial_sols_23_arr = find_nontrivial_solutions(A_23, f_23)
                
            valid_sols_23 = create_four_equation_for_23(
                input_matrix_octal, nontrivial_sols_23_arr, current_sol_12
            )

            relations.add_related_array(x12_idx, valid_sols_23)

        print("\nШаг 3: Поиск решений x_31...")
        
        for x12_idx, current_sol_12 in enumerate(valid_sols_12):
            related_sols_23 = relations.get_related_arrays(x12_idx)
                
            for x23_idx, current_sol_23 in enumerate(related_sols_23):
                system_31_binary = create_linear_system_for_31(
                    system_23_binary, current_sol_12, current_sol_23, input_matrix_octal
                )
                A_31 = Matrix(GF(2), system_31_binary)
                f_31 = vector(GF(2), [0]*(A_31.nrows()-1) + [1])
                
                nontrivial_sols_31_arr = find_nontrivial_solutions(A_31, f_31)
                    
                valid_sols_31 = create_five_equation_for_31(
                    input_matrix_octal, current_sol_23, current_sol_12, nontrivial_sols_31_arr
                )
                
                relations.add_sub_related_array(x12_idx, x23_idx, valid_sols_31)
        
                print("\nШаг 4: Поиск решений x_13...")
            for x23_idx, current_sol_23 in enumerate(related_sols_23):
                x31_entries = relations.get_sub_related_arrays(x12_idx, x23_idx)        
                
                for x31_idx, x31_entry in enumerate(x31_entries):
                    current_sol_31 = x31_entry['array_x31']
                    
                    system_13_binary = create_linear_system_for_13(
                        system_31_binary, current_sol_12, current_sol_23, current_sol_31, input_matrix_octal
                    )
                    A_13 = Matrix(GF(2), system_13_binary)
                    f_13 = vector(GF(2), [0]*27 + [1]*2 + [0]*7)
                    
                    nontrivial_sols_13_arr = find_nontrivial_solutions(A_13, f_13)
                    if not nontrivial_sols_13_arr.size:
                        continue
                        
                    valid_sols_13 = create_six_equation_for_13(
                        input_matrix_octal, current_sol_31, current_sol_23, current_sol_12, nontrivial_sols_13_arr
                    )
                    
                    if valid_sols_13:
                        print(f"!!! НАЙДЕНО РЕШЕНИЕ x13 !!!")
                        return True, {
                            "input_matrix": input_matrix_octal,
                            "x12": current_sol_12,
                            "x23": current_sol_23,
                            "x31": current_sol_31,
                            "x13": valid_sols_13[0]
                        }
        
        return False, "x_13 не найден после всех проверок"
        
    except Exception as e:
        print(f"Ошибка в run_pipeline_for_one_matrix: {type(e).__name__} - {str(e)}")
        traceback.print_exc()
        return False, f"Ошибка: {type(e).__name__} - {str(e)}"

In [ ]:
precompute_dicts()

found_matrices_and_solutions = []
current_flat_octal_iter = None 

max_matrices_to_check = 1000000
matrices_generated_count = 0
matrices_processed_after_filter = 0
max_successful_finds = 5

fixed_trace_matrix_rows = [
    [7, 7, 7], 
    [7, 2, 5], 
    [7, 1, 6]  
]

# Для отладки можно начать с конкретной матрицы:
#test_matrix = [[7, 6, 4, 2, 1, 6, 6], [7, 1, 3, 1, 2, 5, 5], [7, 3, 5, 1, 2, 3, 3]] # Пример
#test_flat = [d for row in test_matrix for d in row]
#for i in range(21): # "Откатить" на одну, чтобы генератор выдал ее первой
#    test_flat[20-i] -=1
#    if test_flat[20-i] >= 0: 
#        break
#    else: test_flat[20-i] = 7
#current_flat_octal_iter = test_flat



INFO: Предварительный расчет словарей G_dict и F_dict...
INFO: Словари G_dict и F_dict рассчитаны.


In [ ]:
max_zero_columns_allowed = 2
print(f"INFO: Запуск системы поиска матриц (до {max_matrices_to_check} проверок или {max_successful_finds} находок)...")
start_total_time = time.time()

while (matrices_generated_count < max_matrices_to_check and 
        len(found_matrices_and_solutions) < max_successful_finds):
    
    input_matrix, next_flat_octal_for_iter = generate_next_complement_matrix_octal(current_flat_octal_iter)
    
    if input_matrix is None:
        print("INFO: Все возможные матрицы были перебраны генератором.")
        break
    
    current_flat_octal_iter = next_flat_octal_for_iter
    matrices_generated_count += 1
        
    full_input_matrix = [
        fixed_trace_matrix_rows[0] + input_matrix[0],
        fixed_trace_matrix_rows[1] + input_matrix[1],
        fixed_trace_matrix_rows[2] + input_matrix[2]
    ]
    
    zero_cols = count_zero_columns(full_input_matrix)    
    if zero_cols > max_zero_columns_allowed:
        continue

    satisfied_condition = check_conditions_on_complement_cols_v7(full_input_matrix)

    if satisfied_condition:
        #print(f"DEBUG: Матрица {input_matrix} (условие: {satisfied_condition}) проходит фильтр. Обработка...")
        matrices_processed_after_filter += 1
        
        success, data_or_msg = run_pipeline_for_one_matrix(full_input_matrix) 
        
        if success:
            data_or_msg['satisfied_condition_type'] = satisfied_condition 
            found_matrices_and_solutions.append(data_or_msg)
            print(f"!!! УСПЕХ ({len(found_matrices_and_solutions)}/{max_successful_finds}) !!! Найдена матрица ({satisfied_condition}): {data_or_msg.get('input_matrix')}")
        else:

            print(f"INFO: Для {full_input_matrix} ({satisfied_condition}) решения x_13 не найдены. Причина: {data_or_msg}")
            pass
    
    if matrices_generated_count % 10000 == 0 or \
        (matrices_generated_count < 1000 and matrices_generated_count % 100 == 0) or \
        (matrices_generated_count < 100 and matrices_generated_count % 10 == 0) :
        elapsed_time = time.time() - start_total_time
        rate = matrices_generated_count / elapsed_time if elapsed_time > 0 else 0
        print(f"INFO: Сгенерировано: {matrices_generated_count}. "
                f"Прошло фильтр: {matrices_processed_after_filter}. "
                f"Найдено: {len(found_matrices_and_solutions)}. "
                f"Время: {elapsed_time:.2f}с. Скорость: {rate:.2f} матр/с.")

end_total_time = time.time()
print("\n--- ИТОГОВЫЕ РЕЗУЛЬТАТЫ ---")
print(f"Всего сгенерировано матриц: {matrices_generated_count}")
print(f"Прошло фильтр и было обработано: {matrices_processed_after_filter}")
print(f"Найдено матриц с решениями x_13: {len(found_matrices_and_solutions)}")
print(f"Общее время выполнения: {end_total_time - start_total_time:.2f} секунд.")
if found_matrices_and_solutions:
    for i, result in enumerate(found_matrices_and_solutions):
        print(f"\nУспешный набор #{i+1} (Условие: {result.get('satisfied_condition_type', 'N/A')}):")
        print(f"  Входная матрица: {result.get('input_matrix')}")
        print(f"  x12: {result.get('x12')}")
        print(f"  x23: {result.get('x23')}")
        print(f"  x31: {result.get('x31')}")
        print(f"  x13: {result.get('x13')}")
else:
    print("Подходящих матриц с полным набором решений не найдено в рамках заданных ограничений.")

INFO: Запуск системы поиска матриц (до 1000000 проверок или 5 находок)...

=== Начинаем обработку матрицы: [[7, 7, 7, 0, 0, 0, 0], [7, 2, 5, 0, 0, 0, 0], [7, 1, 6, 0, 0, 1, 1]] ===

Шаг 1: Поиск решений x_12...
Найдено нетривиальных решений x_12: 16383
Валидных решений x_12 после проверки: 1927

Шаг 2: Поиск решений x_23...

Шаг 3: Поиск решений x_31...
Ошибка в run_pipeline_for_one_matrix: ValueError - matrix equation has no solutions


Traceback (most recent call last):
  File "/var/folders/31/r2q_t_c95h16kh185d313c5r0000gn/T/ipykernel_35679/2053847716.py", line 231, in run_pipeline_for_one_matrix
    nontrivial_sols_31_arr = find_nontrivial_solutions(A_31, f_31)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/31/r2q_t_c95h16kh185d313c5r0000gn/T/ipykernel_35679/2331299485.py", line 34, in find_nontrivial_solutions
    x0 = A.solve_right(f)
         ^^^^^^^^^^^^^^^^
  File "sage/matrix/matrix2.pyx", line 956, in sage.matrix.matrix2.Matrix.solve_right (build/cythonized/sage/matrix/matrix2.c:16478)
    X = self._solve_right_general(C, check=check)
  File "sage/matrix/matrix2.pyx", line 1075, in sage.matrix.matrix2.Matrix._solve_right_general (build/cythonized/sage/matrix/matrix2.c:17999)
    raise ValueError("matrix equation has no solutions")
ValueError: matrix equation has no solutions


INFO: Для [[7, 7, 7, 0, 0, 0, 0], [7, 2, 5, 0, 0, 0, 0], [7, 1, 6, 0, 0, 1, 1]] (C1_доп_AABB_или_AAAA) решения x_13 не найдены. Причина: Ошибка: ValueError - matrix equation has no solutions
INFO: Сгенерировано: 10. Прошло фильтр: 1. Найдено: 0. Время: 1843.14с. Скорость: 0.01 матр/с.

=== Начинаем обработку матрицы: [[7, 7, 7, 0, 0, 0, 0], [7, 2, 5, 0, 0, 0, 0], [7, 1, 6, 0, 0, 1, 2]] ===

Шаг 1: Поиск решений x_12...
Найдено нетривиальных решений x_12: 16383
Валидных решений x_12 после проверки: 1927

Шаг 2: Поиск решений x_23...


KeyboardInterrupt: 